### Coding out a recurrent neural network and LSTM model 

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime
import sys

#### Activation_functions 

   The activation functions used are the tanh and the softmax activation functions as the tanh activation has in providing non-linearity and softmax is used at the output so that all the output values sum to 1. (Since we are dealing with a sparse indexed output).

In [2]:
def tanh_activation(Z):
     return (np.exp(Z)-np.exp(-Z))/(np.exp(Z)-np.exp(-Z)) # this is the tanh function can also be written as np.tanh(Z)
def softmax_activation(Z):
        e_x = np.exp(Z - np.max(Z))  # this is the code for softmax function 
        return e_x / e_x.sum(axis=0) 

So now the delta cross entropy, so what we have here is the dL/dO which is the first part of the chain rule as described in the above block. So the calculation done here is that, the predicted output is sub tracted from the particular word index in the one hot encoded format. So its like saying  where the i is the index of the right word ( which is known by having the value 1). (y_i -1 ) -> This is the derivitve if the Loss function cross-entropy and the softrmax activation combined. 

In [3]:
def delta_cross_entropy(predicted_output,original_t_output):
    li = []
    grad = predicted_output
    for i,l in enumerate(original_t_output): #check if the value in the index is 1 or not, if yes then take the same index value from the predicted_ouput list and subtract 1 from it. 
        if l == 1:
    #grad = np.asarray(np.concatenate( grad, axis=0 ))
            grad[i] -= 1
    return grad

Provide the tanh derivitve link 

In [4]:
def tanh_activation_backward(x,top_diff):
    output = np.tanh(x)
    return (1.0 - np.square(output)) * top_diff

#### Gradient calculation function

Here we have to take care about two things, one is that we have to take derivatives with respect to the weight and the other with respect to the previous function the particular weight multiplies with eg : let us consider yt_unactivated, it is the multiplication of output weights and the ht_activated. So from this we can say that we required a gradient for  output_weights as that have to be learnt, so that is represented by dV, now the other gradient belongs to the chain rule that was required to finf the gradient for input_weights , which is dL/dW. So that why the below function in multiplication returns two derivatives. Add the relevent image to this.

In [5]:
def multiplication_backward(weights,x,dz):
    gradient_weight = np.array(np.dot(np.asmatrix(dz),np.transpose(np.asmatrix(x))))
    chain_gradient = np.dot(np.transpose(weights),dz)
    return gradient_weight,chain_gradient
def add_backward(x1,x2,dz):
    dx1 = dz * np.ones_like(x1)
    dx2 = dz * np.ones_like(x2)
    return dx1,dx2

#### Input and weight parameters 

This is the cell wehre we initialize the parameters

In [6]:
learning_rate = 0.0001    
nepoch = 25               
T = 4   # length of sequence
outlook = 1
hidden_dim = 100       
output_dim = 80 # this is the total unique words in the vocabulary
bptt_truncate = 2 # here instead of using vanilla back-propogation, we use a truncated back propogation, so while doing BP we look back at atmost 2 cells 

These are the weights initialization, random weight values ae generated between 0 and 1.

In [7]:
input_weights = np.random.uniform(0, 1, (hidden_dim,hidden_dim))
activation_weights = np.random.uniform(0,1, (hidden_dim, hidden_dim))
output_weights = np.random.uniform(0,1, (output_dim,hidden_dim))
prev_memory =  np.random.uniform(0,1, (hidden_dim,outlook))

Since we need the gradient values updating the weights, they are initialized wth the same shape sized of their respective weights. 

In [8]:
dU = np.zeros(input_weights.shape)
dV = np.zeros(output_weights.shape)
dW = np.zeros(activation_weights.shape)

So instead of taking just one hot encoded vectors for each word like the image,  I have tried out using a word embedding scenario, where I have initialized embedding vectors of 100x1 shape.  

In [9]:
embeddings = []
for i in range(0,T):
    x = np.random.randn(hidden_dim,1)
    embeddings.append(x)

In the below code I have considered an identity matrix, as in a real word secnario, an index based vocabulary approach.

In [10]:
input_string = [2,45,10,65]
output_string = [45,10,65,1]
identity_matrix = np.identity(80, dtype = float)

The output mapper is a dictionary that index of the word and the one hot encoded vector of the word being present at that index. You print the output mapper to see how it looks.

In [11]:
output_mapper = {}
for index_value in output_string :
    output_mapper[index_value]  = identity_matrix[index_value,:]

In [12]:
output_mapper

{45: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 10: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 65: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

In [13]:
output_t = {}
i=0
for key,value in output_mapper.items():
    output_t[i] = value
    i+=1;

In [14]:
output_t

{0: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 2: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.,

### Coding the RNN_box

 The Rnn box is the one that has been used 

In [15]:
def Rnn_forward(input_embedding, input_weights, activation_weights, prev_memory,output_weights):
    forward_params = []
    U_frd = np.dot(activation_weights,prev_memory)
    W_frd = np.asarray(np.dot(input_weights,input_embedding))
    sum_s = W_frd + U_frd
    ht_activated = tanh_activation(sum_s)
    yt_unactivated = np.asarray(np.dot(output_weights,  tanh_activation(sum_s)))
    yt_activated = softmax_activation(yt_unactivated)
    forward_params.append([W_frd,U_frd,sum_s,yt_unactivated])
    return ht_activated,yt_activated,forward_params

In [16]:
def full_forward_prop(T, embeddings ,input_weights,activation_weights,prev_memory,output_weights):
    output_strings = []
    memory = {}
    prev_ht_activation = prev_memory
    for t in range(0,T):
        curr_activation, curr_output, params = Rnn_forward(embeddings[t], input_weights, activation_weights, prev_memory,output_weights)
        output_strings.append(curr_output)
        prev_ht_activation = curr_activation
        memory["ht" + str(t)] = prev_ht_activation
        memory["yt" + str(t)] = curr_output
        memory["params" + str(t)] = params
    return output_strings, memory 

In [17]:
def calculate_loss(output_mapper,predicted_output):
    total_loss = 0
    layer_loss = []
    for y,y_ in zip(output_mapper.values(),predicted_output): # this for loop calculation is for the first equation, where loss for each time-stamp is calculated
        loss = -sum(y[i]*np.log2(y_[i]) for i in range(len(y)))
        loss = loss/ float(len(y))
        layer_loss.append(loss) 
    for i in range(len(layer_loss)): #this the total loss calculated for all the time-stamps considered together. 
        total_loss  = total_loss + layer_loss[i]
    return total_loss/float(len(predicted_output))
    

 ### Backpropogation

In [18]:
def single_backprop(X,input_weights,activation_weights,output_weights,ht_activated,dLo,forward_params_t,diff_s,prev_s):# inlide all the param values for all the data thats there
    W_frd = forward_params_t[0][0] 
    U_frd = forward_params_t[0][1]
    ht_unactivated = forward_params_t[0][2]
    yt_unactivated = forward_params_t[0][3]
    dV,dsv = multiplication_backward(output_weights,ht_activated,dLo)
    ds = np.add(dsv,diff_s) # used for truncation of memory 
    dadd = tanh_activation_backward(ht_unactivated, ds)
    dmulw,dmulu = add_backward(U_frd,W_frd,dadd)
    dW, dprev_s = multiplication_backward(activation_weights, prev_s ,dmulw)
    dU, dx = multiplication_backward(input_weights, X, dmulu) #input weights
    return (dprev_s, dU, dW, dV)

Truncate is used to see how many previous layers we require to compute the gradient. 

In [19]:
def rnn_backprop(embeddings,memory,output_t,dU,dV,dW,bptt_truncate,input_weights,output_weights,activation_weights):
    T = 4
    # we start the backprop from the first timestamp. 
    for t in range(4):
        prev_s_t = np.zeros((hidden_dim,1)) #required as the first timestamp does not have a previous activation or memory
        diff_s = np.zeros((hidden_dim,1)) # this is used for the truncating purpose of restoring a previous information from the before level
        predictions = memory["yt" + str(t)]
        ht_activated = memory["ht" + str(t)]
        forward_params_t = memory["params"+ str(t)] 
        dLo = delta_cross_entropy(predictions,output_t[t]) #the loss derivative for that particular timestamp
        dprev_s, dU_t, dW_t, dV_t = single_backprop(embeddings[t],input_weights,activation_weights,output_weights,ht_activated,dLo,forward_params_t,diff_s,prev_s_t)
        prev_s_t = ht_activated
        prev = t-1
        dLo = np.zeros((output_dim,1)) #here the loss deriative is turned to 0 as we do not require it for the turncated information.
        # the following code is for the trunated bptt and its for each time-stamp. 
        for i in range(t-1,max(-1,t-bptt_truncate),-1):
            forward_params_t = memory["params" + str(i)]
            ht_activated = memory["ht" + str(i)]
            prev_s_i = np.zeros((hidden_dim,1)) if i == 0 else memory["ht" + str(prev)]
            dprev_s, dU_i, dW_i, dV_i = single_backprop(embeddings[t] ,input_weights,activation_weights,output_weights,ht_activated,dLo,forward_params_t,dprev_s,prev_s_i)
            dU_t += dU_i #adding the previous gradients on lookback to the current time sequence 
            dW_t += dW_i
        dV += dV_t 
        dU += dU_t
        dW += dW_t
    return (dU, dW, dV)
    

### Weights updation

In [23]:
def sgd_step(learning_rate, dU,dW,dV, input_weights, activation_weights,output_weights ):
    input_weights -= learning_rate* dU
    activation_weights -= learning_rate * dW
    output_weights -=learning_rate * dV
    return input_weights,activation_weights,output_weights

### Training the sequence

In [24]:
def train(T, embeddings,output_t,output_mapper,input_weights,activation_weights,output_weights,dU,dW,dV,prev_memory,learning_rate=0.001, nepoch=100, evaluate_loss_after=2):
    losses = []
    for epoch in range(nepoch):
        if(epoch % evaluate_loss_after == 0):
                output_string,memory = full_forward_prop(T, embeddings ,input_weights,activation_weights,prev_memory,output_weights)
                loss = calculate_loss(output_mapper, output_string)
                losses.append(loss)
                time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                print("%s: Loss after  epoch=%d: %f" % (time,epoch, loss))
                sys.stdout.flush()
        dU,dW,dV = rnn_backprop(embeddings,memory,output_t,dU,dV,dW,bptt_truncate,input_weights,output_weights,activation_weights)
        input_weights,activation_weights,output_weights= sgd_step(learning_rate,dU,dW,dV,input_weights,activation_weights,output_weights)
    return losses

In [25]:
losses = train(T, embeddings,output_t,output_mapper,input_weights,activation_weights,output_weights,dU,dW,dV,prev_memory,learning_rate=0.0001, nepoch=10, evaluate_loss_after=2)

2020-09-05 19:59:35: Loss after  epoch=0: 0.135151
2020-09-05 19:59:35: Loss after  epoch=2: 0.133580
2020-09-05 19:59:35: Loss after  epoch=4: 0.130551
2020-09-05 19:59:35: Loss after  epoch=6: 0.126094
2020-09-05 19:59:35: Loss after  epoch=8: 0.120250


In [ ]:
losses